In [1]:
%load_ext autoreload
%autoreload 2

import os
import sys

import nest_asyncio


sys.path.insert(0, os.path.abspath('..'))
nest_asyncio.apply()

In [2]:
import logging


# TODO where to put logging ocnfig?
logging.basicConfig(
    level=logging.INFO, format='%(asctime)s - %(levelname)s - %(message)s'
)

In [3]:
from decouple import config

from math_rag.repositories.documents import MongoDocumentRepository
from math_rag.repositories.embeddings import QdrantEmbeddingRepository
from math_rag.repositories.files import GoogleFileRepository
from math_rag.repositories.graphs import Neo4jGraphRepository
from math_rag.repositories.objects import MinioObjectRepository


DEPLOYMENT = config('DEPLOYMENT')

MINIO_ENDPOINT = config('MINIO_ENDPOINT')
MINIO_ACCESS_KEY = config('MINIO_ACCESS_KEY')
MINIO_SECRET_KEY = config('MINIO_SECRET_KEY')
MONGO_HOST = config('MONGO_HOST')
NEO4J_URI = config('NEO4J_URI')
NEO4J_USERNAME = config('NEO4J_USERNAME')
NEO4J_PASSWORD = config('NEO4J_PASSWORD')
QDRANT_URL = config('QDRANT_URL')

In [4]:
object_repo = MinioObjectRepository(MINIO_ENDPOINT, MINIO_ACCESS_KEY, MINIO_SECRET_KEY)
embedding_repo = QdrantEmbeddingRepository(QDRANT_URL)
graph_repo = Neo4jGraphRepository(NEO4J_URI, NEO4J_USERNAME, NEO4J_PASSWORD)
document_repo = MongoDocumentRepository(MONGO_HOST, DEPLOYMENT)
file_repo = GoogleFileRepository()

2025-02-23 16:16:09,079 - INFO - HTTP Request: GET http://localhost:6333 "HTTP/1.1 200 OK"


In [5]:
import gzip
import shutil
import tarfile

from pathlib import Path

from pylatexenc.latexwalker import (
    LatexCharsNode,
    LatexCommentNode,
    LatexEnvironmentNode,
    LatexGroupNode,
    LatexMacroNode,
    LatexMathNode,
    LatexNode,
    LatexSpecialsNode,
    LatexWalker,
)


ARTICLES_PATH = '../tmp/articles'

In [6]:
credentials_path = Path('../google/credentials.json')
token_path = Path('../google/token.json')
resource = file_repo.authenticate(credentials_path, token_path)

2025-02-23 16:16:16,113 - INFO - file_cache is only supported with oauth2client<4.0.0


In [11]:
folder_name = 'articles'
file_name = 'articles_v1.zip'  # NOTE

file_id = file_repo.get_file_id(resource, file_name, folder_name)
assert file_id is not None

file_bytes = file_repo.get_file_by_id(resource, file_id)

In [12]:
from io import BytesIO
from zipfile import ZipFile


with ZipFile(file_bytes, 'r') as zip_file:
    files = {
        name: BytesIO(zip_file.read(name))
        for name in zip_file.namelist()
        if not name.endswith('/')
    }

In [13]:
bucket_name = 'articles'
object_repo.create_bucket(bucket_name)

In [ ]:
# object_repo.clear_bucket(bucket_name)

In [32]:
for file_name, file_bytes in files.items():
    object_repo.insert_object(bucket_name, file_name, file_bytes)

In [33]:
for file_name in object_repo.list_object_names(bucket_name):
    print(file_name)

articles_v1/ac/2502.13273v1/2502.13273v1.pdf
articles_v1/ac/2502.13273v1/AlgClos14.tex
articles_v1/ac/2502.13276v1/2502.13276v1.pdf
articles_v1/ac/2502.13276v1/Main.tex
articles_v1/ac/2502.13806v1/2502.13806v1.pdf
articles_v1/ac/2502.13806v1/arXiv.bbl
articles_v1/ac/2502.13806v1/arXiv.tex
articles_v1/ac/2502.13903v1/2502.13903v1.pdf
articles_v1/ac/2502.13903v1/dpsl2_20250219.tex
articles_v1/ag/2502.13800v1/2502.13800v1.pdf
articles_v1/ag/2502.13800v1/neron.tex
articles_v1/ag/2502.13815v1/2502.13815v1.pdf
articles_v1/ag/2502.13815v1/Z3-v3.tex
articles_v1/ag/2502.13882v1/2502.13882v1.pdf
articles_v1/ag/2502.13882v1/main.tex
articles_v1/ag/2502.13892v1/2502.13892v1.pdf
articles_v1/ag/2502.13892v1/_commands.tex
articles_v1/ag/2502.13892v1/_preamble.tex
articles_v1/ag/2502.13892v1/abstract.tex
articles_v1/ag/2502.13892v1/appl.tex
articles_v1/ag/2502.13892v1/cl.tex
articles_v1/ag/2502.13892v1/faces.tex
articles_v1/ag/2502.13892v1/filt.tex
articles_v1/ag/2502.13892v1/intro.tex
articles_v1/ag/

In [ ]:
for file_name in object_repo.list_object_names(bucket_name):
    file_bytes = object_repo.get_object(bucket_name, file_name)

### Extract

In [ ]:
def get_gzip_original_filename(file_path):
    with open(file_path, 'rb') as f:
        if f.read(2) != b'\x1f\x8b':
            return None
        f.read(1)
        flag = f.read(1)[0]
        f.read(4)
        f.read(1)
        f.read(1)
        orig_name = None
        if flag & 0x08:
            name_bytes = bytearray()
            while True:
                b = f.read(1)
                if not b or b == b'\x00':
                    break
                name_bytes.extend(b)
            try:
                orig_name = name_bytes.decode('utf-8')
            except UnicodeDecodeError:
                orig_name = name_bytes.decode('latin1')
        return orig_name


def extract_gz(file_path, dest_folder):
    orig_name = get_gzip_original_filename(file_path)
    if not orig_name:
        orig_name = os.path.splitext(os.path.basename(file_path))[0]
    dest_path = os.path.join(dest_folder, orig_name)
    with gzip.open(file_path, 'rb') as f_in, open(dest_path, 'wb') as f_out:
        shutil.copyfileobj(f_in, f_out)


def extract_tar_gz(file_path, dest_folder):
    with tarfile.open(file_path, 'r:gz') as tar:
        tar.extractall(path=dest_folder)


def process_subdir(subdir_path):
    files = os.listdir(subdir_path)
    pdf_files = {f for f in files if f.endswith('.pdf')}
    for pdf in pdf_files:
        base_name = pdf[:-4]
        gz_name = f'arXiv-{base_name}.gz'
        tar_gz_name = f'arXiv-{base_name}.tar.gz'
        gz_file = None
        if tar_gz_name in files:
            gz_file = tar_gz_name
        elif gz_name in files:
            gz_file = gz_name
        if gz_file:
            new_dir = os.path.join(subdir_path, base_name)
            os.makedirs(new_dir, exist_ok=True)
            shutil.move(os.path.join(subdir_path, pdf), new_dir)
            shutil.move(os.path.join(subdir_path, gz_file), new_dir)
            new_gz_path = os.path.join(new_dir, gz_file)
            if gz_file.endswith('.tar.gz'):
                extract_tar_gz(new_gz_path, new_dir)
            else:
                extract_gz(new_gz_path, new_dir)


def extract_all():
    for subdir in os.listdir(ARTICLES_PATH):
        subdir_path = os.path.join(ARTICLES_PATH, subdir)
        if os.path.isdir(subdir_path):
            process_subdir(subdir_path)


extract_all()

/tmp/ipykernel_15043/154124979.py:39: DeprecationWarning: Python 3.14 will, by default, filter extracted tar archives and reject files or modify their metadata. Use the filter argument to control this behavior.
  tar.extractall(path=dest_folder)


In [9]:
def clean():
    for root, dirs, files in os.walk(ARTICLES_PATH):
        for file in files:
            if file.endswith('.gz'):
                os.remove(os.path.join(root, file))


clean()

### Categorize

In [ ]:
from enum import Enum


class ArXivMathCategory(str, Enum):
    AC = 'commutative_algebra'
    AG = 'algebraic_geometry'
    AP = 'analysis_of_pdes'
    AT = 'algebraic_topology'
    CA = 'classical_analysis_and_odes'
    CO = 'combinatorics'
    CT = 'category_theory'
    CV = 'complex_variables'
    DG = 'differential_geometry'
    DS = 'dynamical_systems'
    FA = 'functional_analysis'
    GM = 'general_mathematics'
    GN = 'general_topology'
    GR = 'group_theory'
    GT = 'geometric_topology'
    HO = 'history_and_overview'
    IT = 'information_theory'
    KT = 'k_theory_and_homology'
    LO = 'logic'
    MG = 'metric_geometry'
    MP = 'mathematical_physics'
    NA = 'numerical_analysis'
    NT = 'number_theory'
    OA = 'operator_algebras'
    OC = 'optimization_and_control'
    PR = 'probability'
    QA = 'quantum_algebra'
    RA = 'rings_and_algebras'
    RT = 'representation_theory'
    SG = 'symplectic_geometry'
    SP = 'spectral_theory'
    ST = 'statistics_theory'

### Parse

In [5]:
def read_tex_file(file_path: Path) -> tuple[str, str]:
    for encoding in ('utf-8', 'latin1', 'cp1252'):
        try:
            with open(file_path, 'r', encoding=encoding) as file:
                return file.read(), encoding

        except UnicodeDecodeError:
            continue

In [ ]:
from pylatexenc.macrospec import ParsedMacroArgs


def visit_nodes_old(file_path: Path):
    latex, encoding = read_tex_file(file_path)
    walker = LatexWalker(latex)
    nodes, _, _ = walker.get_latex_nodes()

    stack = [iter(nodes)]

    while stack:
        node = next(stack[-1], None)

        if node is None:
            stack.pop()
            continue

        if isinstance(node, LatexCharsNode):
            pass

        elif isinstance(node, LatexMathNode):
            pass

        elif isinstance(node, LatexCommentNode):
            pass

        elif isinstance(node, LatexSpecialsNode):
            pass

        elif isinstance(node, LatexMacroNode):
            # if node.macroname == 'input':
            #     args: ParsedMacroArgs = node.nodeargd
            #     assert len(args.argnlist) == 1

            #     group_node = args.argnlist[0]
            #     if not isinstance(group_node, LatexGroupNode):
            #         continue

            #     assert len(group_node.nodelist) == 1

            #     chars_node: LatexCharsNode = group_node.nodelist[0]
            #     input_file_name = str(chars_node.chars)
            #     input_file_path = dir_path / input_file_name

            #     if not input_file_path.suffix:
            #         input_file_path = input_file_path.with_suffix('.tex')

            #     if input_file_path.suffix == '.tex':
            #         latex, encoding = read_tex_file(input_file_path)
            #         walker = LatexWalker(latex)
            #         nodes, _, _ = walker.get_latex_nodes()
            #         stack.append(iter(nodes))

            # elif node.macroname == 'include':
            #     pass
            pass

        elif isinstance(node, LatexEnvironmentNode):
            stack.append(iter(node.nodelist))

        elif isinstance(node, LatexGroupNode):
            stack.append(iter(node.nodelist))

In [6]:
from typing import Callable, Iterator


def visit_nodes(
    file_path: Path, callbacks: dict[type[LatexNode], Callable[[LatexNode], None]]
):
    latex, _ = read_tex_file(file_path)
    walker = LatexWalker(latex)
    nodes, _, _ = walker.get_latex_nodes()

    stack: list[Iterator[LatexNode]] = [iter(nodes)]

    while stack:
        node = next(stack[-1], None)

        if node is None:
            stack.pop()
            continue

        node_type = type(node)

        if node_type in callbacks:
            callbacks[node_type](node)

        if isinstance(node, LatexEnvironmentNode) or isinstance(node, LatexGroupNode):
            stack.append(iter(node.nodelist))

In [7]:
# NOTE:
# problem with multiple .tex files -> which one is root? -> one with doc env node?
# problem with relative \input{} -> root dir needed, i.e., .../2502.13810v1

math_nodes: list[LatexMathNode] = []
append_math_node = lambda x: math_nodes.append(x)

callbacks = {LatexMathNode: append_math_node}

for file_path in Path(ARTICLES_PATH).rglob('*.tex'):
    visit_nodes(file_path, callbacks)

len(math_nodes)

127133

In [ ]:
from uuid import UUID, uuid4

from pydantic import BaseModel, Field


class MathExpressionRecord(BaseModel):
    id: UUID = Field(default_factory=uuid4)
    latex: str
    position: int
    is_inline: bool

In [140]:
def fix_latex(latex_str: str):
    fixed = latex_str.replace('\\[', '$$').replace('\\]', '$$')
    fixed = fixed.replace('\\EE', '\\mathbb{E}')
    fixed = fixed.replace('\\II', '\\mathbb{I}')
    fixed = fixed.replace('\\Var', '\\mathrm{Var}')
    fixed = fixed.replace('\\HH', '\\mathbb{H}')
    fixed = fixed.replace('\\AND', '\\wedge')
    fixed = fixed.replace('\\OR', '\\vee')
    fixed = fixed.replace('\\mathbbm{1}', '\\mathbf{1}')
    fixed = fixed.replace('\\Maj', '\\mathrm{Maj}')
    fixed = fixed.replace('\\sgn', '\\operatorname{sgn}')
    fixed = fixed.replace('\\Tribus', '\\mathrm{Tribus}')
    fixed = fixed.replace('\\linebreak', '\\text{ }')
    fixed = fixed.replace('\\Prob', '\\mathbb{P}')
    fixed = fixed.replace('\\WW', '\\mathcal{W}')

    return fixed

In [ ]:
from IPython.display import Math, display


for i, latex_math_node in enumerate(math_nodes[:100]):
    latex = latex_math_node.latex_verbatim()
    latex_fixed = fix_latex(latex)
    math_display_object = Math(latex_fixed)

    # print(i)
    # print(math_display_object._repr_latex_())

    display(math_display_object)

### Classify

In [2]:
from decouple import config

from math_rag.inference.llms import LLM


OPENAI_BASE_URL = config('OPENAI_BASE_URL')
OPENAI_API_KEY = config('OPENAI_API_KEY')

In [3]:
llm = LLM(model='gpt-4o-mini', base_url=OPENAI_BASE_URL, api_key=OPENAI_API_KEY)

In [55]:
math_expr = math_nodes[13].latex_verbatim()  # 13
math_expr

'$\\mu(x)=\\frac{1}{2^n}$'

In [56]:
from openai import NOT_GIVEN


prompt = f"""
You are a mathematical expression classifier.
Given a mathematical expression, classify it in one of 4 given classes:
- constant
- variable
- formula
- other

Return a class only!

Mathematical expression:
{math_expr}

Class:
"""

use_json = False
completion = await llm.client.chat.completions.create(
    model=llm.model,
    messages=[{'role': 'user', 'content': prompt}],
    response_format={'type': 'json_object'} if use_json else NOT_GIVEN,
    logprobs=True,
    temperature=0.0,
    top_logprobs=5,
)
print(completion)

ChatCompletion(id='chatcmpl-B3qibSoK3Z0LqPtZQjEYKhXO1AjaF', choices=[Choice(finish_reason='stop', index=0, logprobs=ChoiceLogprobs(content=[ChatCompletionTokenLogprob(token='formula', bytes=[102, 111, 114, 109, 117, 108, 97], logprob=-3.1281633e-07, top_logprobs=[TopLogprob(token='formula', bytes=[102, 111, 114, 109, 117, 108, 97], logprob=-3.1281633e-07), TopLogprob(token=' formula', bytes=[32, 102, 111, 114, 109, 117, 108, 97], logprob=-16.125), TopLogprob(token='Formula', bytes=[70, 111, 114, 109, 117, 108, 97], logprob=-16.375), TopLogprob(token='_formula', bytes=[95, 102, 111, 114, 109, 117, 108, 97], logprob=-18.5), TopLogprob(token='form', bytes=[102, 111, 114, 109], logprob=-20.0)])], refusal=None), message=ChatCompletionMessage(content='formula', refusal=None, role='assistant', audio=None, function_call=None, tool_calls=None))], created=1740257333, model='gpt-4o-mini-2024-07-18', object='chat.completion', service_tier='default', system_fingerprint='fp_7fcd609668', usage=Comple

In [58]:
completion.choices[0].message.content

'formula'

In [57]:
import numpy as np


for x in completion.choices[0].logprobs.content:
    for y in x.top_logprobs:
        print(f'"{y.token}": {np.exp(y.logprob)}')

    print('------')
    print(x.token)
    print(x.logprob)
    print(np.exp(x.logprob))

"formula": 0.9999996871837189
" formula": 9.931194312156244e-08
"Formula": 7.734421907141565e-08
"_formula": 9.237449661970594e-09
"form": 2.061153622438558e-09
------
formula
-3.1281633e-07
0.9999996871837189


In [ ]:
# TODO
# - description for each class
# - how to determine classes?
# - do names need to take a single token?

In [ ]:
class MathExpressionCategory(str, Enum):
    CONSTANT = 'constant'